In [2]:
#import BeautifulSoup

import requests
import bs4
from bs4 import BeautifulSoup

In [3]:
# Request from URL
r = requests.get('http://www.indeed.com/jobs?q=data+scientist&l=New+York+State&start=70')
r

<Response [200]>

In [4]:
#print headers and content

print r.headers
print r.content

{'X-Content-Type-Options': 'nosniff', 'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked', 'Set-Cookie': 'CTK=1avh6o1fm50i98lt; Domain=.indeed.com; Expires=Tue, 25-Oct-2033 08:57:03 GMT; Path=/, ctkgen=1; Expires=Thu, 20-Oct-2016 14:11:32 GMT; Path=/, JSESSIONID=203AA2E7F11EDFCD16C5AA22B1E23B6C.jasxA_iad-job27; Path=/; HttpOnly, INDEED_CSRF_TOKEN=xPpDPH55vqVVuRy4Fq5OESTX1Sj2A4NE; Path=/, PREF=""; Domain=.indeed.com; Expires=Thu, 01-Jan-1970 00:00:10 GMT; Path=/, PREF="TM=1476972512761:L=New+York+State"; Version=1; Max-Age=536870911; Expires=Tue, 25-Oct-2033 08:57:03 GMT; Path=/, RQ="q=data+scientist&l=New+York+State"; Version=1; Max-Age=2592000; Expires=Sat, 19-Nov-2016 14:08:32 GMT; Path=/, UD="LA=1476972512:CV=1476972512:TS=1476972512:SG=3ec294ba6a3217f1c919465c1629ea80"; Version=1; Max-Age=536870911; Expires=Tue, 25-Oct-2033 08:57:03 GMT; Path=/, BIGipServerjob_iad=!j5CQpobx/vj5GNoRNhs+rOzpOInz88uInugiklY63q+583h4fvsTS25NmA9IIl6ytavCFnG+48M6XhE=; path=/', 'Vary': 'User-Agent,

In [5]:
# Use BeautifulSoup

insoup = BeautifulSoup(r.content,'lxml')

In [6]:
# function for getting location from indeed

def extract_location_from_result(loc):
    try:
        for n in loc.findAll(class_='location'):
            return n.text.strip()
    except:
        return 'None'

In [7]:
extract_location_from_result(insoup)

u'New York, NY'

In [8]:
# function for getting job title from indeed

def extract_jobtitle_from_result(jobtitle):
    try:
        for n in jobtitle.findAll(class_='jobtitle'):
            return n.text.strip()
    except:
        return 'None' 

In [9]:
extract_jobtitle_from_result(insoup)

u'Machine Learning Quantitative Analyst'

In [10]:
# function for getting company from indeed

def extract_company_from_result(comp):
    try:
        for n in comp.findAll(class_='company'):
            return n.text.strip()
    except:
        return 'None' 

In [11]:
extract_company_from_result(insoup)

u'Bloomberg'

In [12]:
# function to get salary from indeed

def extract_salary_from_result(sal):
    try:
        for n in sal.findAll('td', class_='snip'):
            return n.nobr
    except:
        return 'None'  

In [13]:
extract_salary_from_result(insoup)

In [14]:
# function to get summary/description from indeed

def extract_summary_from_result(desc):
    try:
        for n in desc.findAll('span', class_='summary'):
            return n.text.strip()
    except:
        return 'None' 

In [15]:
extract_summary_from_result(insoup)

u'The Machine Learning Quantitative Analyst will work in Bloomberg\u2019s Enterprise Solutions area and work collaboratively to build a liquidity tool for banks,...'

In [16]:
#Get each list from each page and 

Location = []
Job_Title = []
Company = []
Salary = []
Description = []

url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 1000
cities = ['New+York', 'Chicago', 'Boston', 'San+Francisco', 'Los+Angeles', 'Austin', 'Atlanta']
results = []
for city in set(cities):
    for start in range(0, max_results_per_city, 10):
        url = url_template.format(city,start)
        ru = requests.get(url)
        indeed = BeautifulSoup(ru.content, "lxml")
        for item in indeed.findAll('div', class_='result'):
            Location.append(extract_location_from_result(item))
            Job_Title.append(extract_jobtitle_from_result(item))
            Company.append(extract_company_from_result(item))
            Salary.append(extract_salary_from_result(item))
            Description.append(extract_summary_from_result(item))

In [17]:
import pandas as pd

In [18]:
df = pd.DataFrame([Location,Job_Title,Company,Salary,Description]).T 

In [19]:
df.columns= ['Location','Job_Title','Company','Salary','Description']

In [20]:
df.head()

,Location,Job_Title,Company,Salary,Description
0,"Austin, TX 78731",Data Engineer,Indeed,None,Work closely with stakeholders on the data dem...
1,"Austin, TX 78759",Senior Data Scientist - Geospatial Analytics,MaxPoint,None,Senior Data Scientist – Geospatial Analytics. ...
2,"Austin, TX 78731",Data Scientist,Indeed,None,How a Data Scientist works. As a Data Scientis...
3,"Austin, TX",Data Scientist,Tiger Analytics,None,As a Data Scientist you will work with our cli...
4,"Austin, TX",Data Scientist,IBM Kenexa,None,The Data Scientist is responsible for the prod...


In [21]:
#Cleaning Salary Column

df['Salary'].unique()

array([None, <nobr>$6,250 - $10,833 a month</nobr>,
       <nobr>$4,599 - $6,066 a month</nobr>, <nobr>$4,584 a month</nobr>,
       <nobr>$5,259 - $8,624 a month</nobr>,
       <nobr>$5,259 - $6,941 a month</nobr>, <nobr>$4,000 a month</nobr>,
       <nobr>$5,400 - $6,500 a month</nobr>,
       <nobr>$140,000 - $160,000 a year</nobr>,
       <nobr>$45,000 - $55,000 a year</nobr>,
       <nobr>$3,520 - $4,600 a month</nobr>, <nobr>$6,667 a month</nobr>,
       <nobr>$5,541 a month</nobr>,
       <nobr>$120,000 - $150,000 a year</nobr>,
       <nobr>$23.44 - $29.80 an hour</nobr>, <nobr>$130,000 a year</nobr>,
       <nobr>$4,500 - $4,900 a month</nobr>, <nobr>$3,519 a month</nobr>,
       <nobr>$90,000 a year</nobr>, <nobr>$60,000 - $80,000 a year</nobr>,
       <nobr>$5,500 - $7,500 a month</nobr>,
       <nobr>$110,000 - $155,000 a year</nobr>,
       <nobr>$120,000 a year</nobr>,
       <nobr>$135,000 - $165,000 a year</nobr>,
       <nobr>$80,000 - $110,000 a year</nobr>,
       <n

In [22]:
# Remove columns with 'None' in Salary

df = df[df.Salary.notnull()]

In [23]:
#Convert Salary column to a string

df['Salary'] = df['Salary'].apply(lambda x: str(x))

In [24]:
# Remove $ and , in Salary

df['Salary'] = df['Salary'].apply(lambda x: x.replace('$',''))
df['Salary'] = df['Salary'].apply(lambda x: x.replace(',',''))

In [25]:
# Remove nobr tags from Salary

df['Salary'] = df['Salary'].apply(lambda x: x.replace('<nobr>',''))
df['Salary'] = df['Salary'].apply(lambda x: x.replace('</nobr>',''))

In [26]:
# Reset index

df = df.reset_index(drop=True)

In [27]:
# Parse yearly, monthly, hourly salaries

df['hourly_sal'] = df['Salary'].map(lambda x: 1 if 'hour' in x else 0)
df['monthly_sal'] = df['Salary'].map(lambda x: 1 if 'month' in x else 0)
df['yearly_sal'] = df['Salary'].map(lambda x: 1 if 'year' in x else 0)


In [28]:
# Drop monthly and hourly salaries
df_year1 = df[df.monthly_sal != 1]
df_year = df_year1[df.hourly_sal !=1]

/Applications/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


In [29]:
# drop columns hourly_sal, monthly_sal, and yearly_sal
df_year = df_year.drop('hourly_sal', axis=1)
df_year = df_year.drop('monthly_sal', axis=1)
df_year = df_year.drop('yearly_sal', axis=1)

In [30]:
# reset index and set dataframe to df
df = df_year.reset_index(drop=True)

In [31]:
# remove a year and - in Salary column
df["Salary"] = df["Salary"].apply(lambda x: x.replace(" a year",""))
df["Salary"] = df["Salary"].apply(lambda x: x.replace("-",""))


In [32]:
# check for other non-numeric values
df['Salary'].unique()

array(['140000  160000', '45000  55000', '120000  150000', '130000',
       '90000', '60000  80000', '110000  155000', '120000',
       '135000  165000', '80000  110000', '100000  120000', '180000',
       '93257  137114', '110000', '75000  90000', '125000', '140000',
       '70000', '104674  151358', '80000', '50000', '60000  75000',
       '100000  130000', '75000  95000', '75000  100000', '130000  180000',
       '145000', '70000  100000', '200000', '125000  175000',
       '70000  105000', '150000', '100000  150000', '34142', '1250 a week',
       '170000  190000', '150000  190000', '110000  180000', '100000',
       '130000  150000', '54800  70900', '150000  180000',
       '200000  300000', '130000  160000', '67500  85000', '170000',
       '140000  180000', '93000  135000', '100000  135000',
       '120000  160000', '120000  170000', '90000  135000', '60000',
       '47860  67712', '160000', '75000  120000', '90000  130000',
       '110000  170000', '36381  48462', '108900  1610

In [33]:
# remove '1250 a week'

df = df[df.Salary.str.contains('1250') == False]
df['Salary'].unique()

array(['140000  160000', '45000  55000', '120000  150000', '130000',
       '90000', '60000  80000', '110000  155000', '120000',
       '135000  165000', '80000  110000', '100000  120000', '180000',
       '93257  137114', '110000', '75000  90000', '140000', '70000',
       '104674  151358', '80000', '50000', '60000  75000',
       '100000  130000', '75000  95000', '75000  100000', '130000  180000',
       '145000', '70000  100000', '200000', '70000  105000', '150000',
       '100000  150000', '34142', '170000  190000', '150000  190000',
       '110000  180000', '100000', '130000  150000', '54800  70900',
       '150000  180000', '200000  300000', '130000  160000',
       '67500  85000', '170000', '140000  180000', '93000  135000',
       '100000  135000', '120000  160000', '120000  170000',
       '90000  135000', '60000', '47860  67712', '160000', '75000  120000',
       '90000  130000', '110000  170000', '36381  48462', '108900  161000',
       '110000  140000', '52000', '95000', '9

In [34]:
df.head()

,Location,Job_Title,Company,Salary,Description
0,"Austin, TX",Senior Machine Learning Data Scientist,All-In Analytics,140000 160000,"Machine Learning Data Scientist. Forecasting, ..."
1,"Austin, TX 78701 (Downtown area)",JIRA Business System Analyst,JE Compliance,45000 55000,Ability to interact with many different types ...
2,"Austin, TX",Machine Learning Engineer,Volt Workforce Solutions,120000 150000,Data mining competition experience preferred (...
3,"Austin, TX",Sr. .Net Developer,Interactive Resources LLC,130000,"We are looking for a collaborative, self-start..."
4,"Austin, TX",Senior Software Engineer/Team Lead,XOR Data Exchange,90000,XOR is leading a data revolution by giving tra...


In [35]:
import numpy as np

In [36]:
# Create seperate columns for columns with dual salaries (Salary 1 and Salary 2)

df['Salary1'] = df['Salary'].apply(lambda x: x.split()[0])
df['Salary2'] = df['Salary'].apply(lambda x: x.split()[-1])

In [37]:
df.head()

,Location,Job_Title,Company,Salary,Description,Salary1,Salary2
0,"Austin, TX",Senior Machine Learning Data Scientist,All-In Analytics,140000 160000,"Machine Learning Data Scientist. Forecasting, ...",140000,160000
1,"Austin, TX 78701 (Downtown area)",JIRA Business System Analyst,JE Compliance,45000 55000,Ability to interact with many different types ...,45000,55000
2,"Austin, TX",Machine Learning Engineer,Volt Workforce Solutions,120000 150000,Data mining competition experience preferred (...,120000,150000
3,"Austin, TX",Sr. .Net Developer,Interactive Resources LLC,130000,"We are looking for a collaborative, self-start...",130000,130000
4,"Austin, TX",Senior Software Engineer/Team Lead,XOR Data Exchange,90000,XOR is leading a data revolution by giving tra...,90000,90000


In [38]:
# Transform string type to int types

df['Salary1'] = pd.to_numeric(df['Salary1'], errors = 'coerce')
df['Salary2'] = pd.to_numeric(df['Salary2'], errors = 'coerce')

print df['Salary1'].dtype
print df['Salary2'].dtype

int64
float64


In [39]:
# Convert Salary column into mean of dual salaries (Salary 1 & Salary 2)
df['Salary'] = (df["Salary1"] + df["Salary2"])/2

In [40]:
# Delete columns Salary1 and Salary2
del df['Salary1']
del df['Salary2']

In [41]:
# Create city column
df['City'] = df['Location'].apply(lambda x: x.split(',')[0])

In [42]:
df.head()

,Location,Job_Title,Company,Salary,Description,City
0,"Austin, TX",Senior Machine Learning Data Scientist,All-In Analytics,150000.0,"Machine Learning Data Scientist. Forecasting, ...",Austin
1,"Austin, TX 78701 (Downtown area)",JIRA Business System Analyst,JE Compliance,50000.0,Ability to interact with many different types ...,Austin
2,"Austin, TX",Machine Learning Engineer,Volt Workforce Solutions,135000.0,Data mining competition experience preferred (...,Austin
3,"Austin, TX",Sr. .Net Developer,Interactive Resources LLC,130000.0,"We are looking for a collaborative, self-start...",Austin
4,"Austin, TX",Senior Software Engineer/Team Lead,XOR Data Exchange,90000.0,XOR is leading a data revolution by giving tra...,Austin


In [43]:
# Move city to be the first column

df = df[['City', 'Location', 'Job_Title', 'Company', 'Salary','Description']]

In [44]:
df.head()

,City,Location,Job_Title,Company,Salary,Description
0,Austin,"Austin, TX",Senior Machine Learning Data Scientist,All-In Analytics,150000.0,"Machine Learning Data Scientist. Forecasting, ..."
1,Austin,"Austin, TX 78701 (Downtown area)",JIRA Business System Analyst,JE Compliance,50000.0,Ability to interact with many different types ...
2,Austin,"Austin, TX",Machine Learning Engineer,Volt Workforce Solutions,135000.0,Data mining competition experience preferred (...
3,Austin,"Austin, TX",Sr. .Net Developer,Interactive Resources LLC,130000.0,"We are looking for a collaborative, self-start..."
4,Austin,"Austin, TX",Senior Software Engineer/Team Lead,XOR Data Exchange,90000.0,XOR is leading a data revolution by giving tra...


In [45]:
df.to_csv('webscrapingds.csv', encoding='utf-8')

In [46]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.grid_search import GridSearchCV

import patsy

In [47]:
df.describe()

/Applications/anaconda/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,Salary
count,216.000000
mean,119329.120370
std,47484.412074
min,10000.000000
25%,NaN
50%,NaN
75%,NaN
max,250000.000000


In [48]:
pd.set_option('display.max_rows', 500)
df.apply(lambda c: c.value_counts()).T.stack()

City         Alpharetta                                                                                                                                                              1.0
             Atlanta                                                                                                                                                                14.0
             Austin                                                                                                                                                                  6.0
             Belmont                                                                                                                                                                 1.0
             Berkeley                                                                                                                                                                3.0
             Boston                                                        

In [49]:
df['Salary_Y'] = df['Salary'].apply(lambda x: 1 if x > 100000 else 0)

In [50]:
df

,City,Location,Job_Title,Company,Salary,Description,Salary_Y
0,Austin,"Austin, TX",Senior Machine Learning Data Scientist,All-In Analytics,150000.0,"Machine Learning Data Scientist. Forecasting, ...",1
1,Austin,"Austin, TX 78701 (Downtown area)",JIRA Business System Analyst,JE Compliance,50000.0,Ability to interact with many different types ...,0
2,Austin,"Austin, TX",Machine Learning Engineer,Volt Workforce Solutions,135000.0,Data mining competition experience preferred (...,1
3,Austin,"Austin, TX",Sr. .Net Developer,Interactive Resources LLC,130000.0,"We are looking for a collaborative, self-start...",1
4,Austin,"Austin, TX",Senior Software Engineer/Team Lead,XOR Data Exchange,90000.0,XOR is leading a data revolution by giving tra...,0
5,Austin,"Austin, TX",LCMS Certifying Scientist,Lighthouse Recruiting,70000.0,Join our Medical Laboratory Scientists Groups:...,0
6,Chicago,"Chicago, IL","Senior Data Scientist (H20, Python, and R)",Workbridge Associates,132500.0,A major Marketing company based in downtown Ch...,1
7,Chicago,"Chicago, IL",Data Scientist -Fitness/Wellness Firm,Hirewell,120000.0,Bring structure to large quantities of data - ...,1
8,Chicago,"Chicago, IL",Data Scientist – Big Data/Artificial Intelligence,Selby Jennings,150000.0,Data Scientist – Big Data/Artificial Intellige...,1
9,Chicago,"Chicago, IL",Data Scientist,Workbridge Associates,95000.0,Big data with Hadoop is a plus. A well-respect...,0


In [51]:
def get_features(dataframe):
    features = patsy.dmatrix('~ C(City) + C(Job_Title) + C(Company)', df)
    X_df = pd.DataFrame(features, columns=features.design_info.column_names)
    X_df.drop('Intercept', axis=1, inplace=True)
    return X_df

In [52]:
X = get_features(df)

In [53]:
y = df['Salary_Y']

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 77)


In [55]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [56]:
y_pred = model.predict(X_test)
y_score = model.decision_function(X_test)

In [57]:
logreg = LogisticRegression(solver='liblinear')
C_vals = [0.0001, 0.001, 0.01, 0.1, .15, .25, .275, .33, 0.5, .66, 0.75, 1.0, 2.5, 5.0, 10.0, 100.0, 1000.0]
penalties = ['l1','l2']

In [58]:
gs = GridSearchCV(logreg, {'penalty': penalties, 'C': C_vals}, verbose=False, cv=3)
gs.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [0.0001, 0.001, 0.01, 0.1, 0.15, 0.25, 0.275, 0.33, 0.5, 0.66, 0.75, 1.0, 2.5, 5.0, 10.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=False)

In [59]:
gs.best_params_

{'C': 2.5, 'penalty': 'l1'}

In [60]:
gs.best_estimator_

LogisticRegression(C=2.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [61]:
conmat = np.array(confusion_matrix(y_test, y_pred, labels=[1,0]))
confusion = pd.DataFrame(conmat, index=['over_100k', 'under_100k'],
                            columns=['predicted_over100k','predicted_under100k'])

print(confusion)

print classification_report(y_test,y_pred)
roc_auc_score(y_test, y_score)

            predicted_over100k  predicted_under100k
over_100k                   43                    5
under_100k                   8                   17
             precision    recall  f1-score   support

          0       0.77      0.68      0.72        25
          1       0.84      0.90      0.87        48

avg / total       0.82      0.82      0.82        73



0.89874999999999994

In [62]:
""" Plot AUC"""

FPR = dict()
TPR = dict()
ROC_AUC = dict()

# For class 1, find the area under the curve
FPR[1], TPR[1], _ = roc_curve(y_test, y_score)
ROC_AUC[1] = auc(FPR[1], TPR[1])

# Plot of a ROC curve for class 1
plt.figure(figsize=[11,9])
plt.plot(FPR[1], TPR[1], label='ROC curve (area = %0.2f)' % ROC_AUC[1], linewidth=4)
#plt.plot([0, 1], [0, 1], 'k--', linewidth=4)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=18)
plt.ylabel('True Positive Rate', fontsize=18)
plt.title('Receiver operating characteristic for high/low income', fontsize=18)
plt.legend(loc="lower right")
plt.show()

In [63]:
def examine_coefficients(model, df):
   df = pd.DataFrame(
       { 'Coefficient' : model.coef_[0] , 'Feature' : df.columns}
   ).sort_values(by='Coefficient')
   return df[df.Coefficient !=0 ]

examine_coefficients(model, X)

,Coefficient,Feature
144,-0.999662,C(Job_Title)[T.Research Analyst]
225,-0.934804,C(Company)[T.Commonwealth of Massachusetts]
224,-0.884106,C(Company)[T.Columbia University]
14,-0.766162,C(City)[T.Long Island City]
260,-0.724935,C(Company)[T.Los Angeles County Department of ...
15,-0.654419,C(City)[T.Los Angeles]
49,-0.617863,C(Job_Title)[T.Catastrophe Risk Analyst - Quan...
261,-0.589930,C(Company)[T.Los Angeles County Human Resources]
259,-0.582440,C(Company)[T.Lighthouse Recruiting]
9,-0.533527,C(City)[T.Chicago]
